<a href="https://colab.research.google.com/github/phuongghj123/CS114.K21/blob/master/Sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

Saving Sarcasm_Headlines_Dataset_v2.json to Sarcasm_Headlines_Dataset_v2 (4).json


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np 
import pandas as pd 
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize
import gensim
from tqdm import tqdm
import json

In [22]:
import io
dataset = pd.read_json(io.StringIO(uploaded["Sarcasm_Headlines_Dataset_v2.json"].decode("utf-8")), lines = True)
dataset.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [23]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
X = []
y = []

for index, rows in dataset.iterrows():
    line = rows.headline
    line = gensim.utils.simple_preprocess(line)
    line = ' '.join(line)
    stop_words = set(stopwords.words('english')) 
  
    word_tokens = word_tokenize(line) 
  
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    line = filtered_sentence
    line = ' '.join(line)
    
    X.append(line)
    line1 = rows.is_sarcastic
    y.append(line1)
print (X[0])
print (y[0])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
thirtysomething scientists unveil doomsday clock hair loss
1


In [0]:
X_data = X
y_data = y
X_test = X
y_test = y

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_data) 
X_data_tfidf =  tfidf_vect.transform(X_data)

X_test_tfidf =  tfidf_vect.transform(X_test)


Label encoder

In [26]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

encoder.classes_

array([0, 1])

In [27]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

X_train, X_val, y_train, y_val = train_test_split(X_data_tfidf, y_data, test_size=0.2, random_state=42)
classifier = ensemble.RandomForestClassifier()
classifier.fit(X_train, y_train)

train_predictions = classifier.predict(X_train)
val_predictions = classifier.predict(X_val)
test_predictions = classifier.predict(X_test_tfidf)

print("Train accuracy: ", metrics.accuracy_score(train_predictions, y_train))
print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))

Train accuracy:  1.0
Validation accuracy:  0.7617051013277428
Test accuracy:  0.9523393549739684
